In [ ]:
import cv2
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None  # Disable the decompression bomb check
from scipy.stats import wasserstein_distance
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import os
import sys

def calculate_ms_ssim(img1, img2, max_level=5, weights=[0.0448, 0.2856, 0.3001, 0.2363, 0.1333]):
    mssim = []
    for level in range(max_level):
        # Dynamically set win_size based on current image dimensions.
        current_win_size = min(11, img1.shape[0], img1.shape[1])
        if current_win_size % 2 == 0:
            current_win_size -= 1
        if current_win_size < 3:
            break

        # Use channel_axis=-1 for color images.
        if img1.ndim == 3:
            sim = ssim(img1, img2, win_size=current_win_size, channel_axis=-1)
        else:
            sim = ssim(img1, img2, win_size=current_win_size)
        mssim.append(sim)

        if level < max_level - 1:
            img1 = cv2.pyrDown(img1)
            img2 = cv2.pyrDown(img2)
    
    # If fewer scales are computed than the number of weights provided, use only the relevant weights.
    weights = weights[:len(mssim)]
    
    # Compute the weighted geometric mean of the SSIM values.
    ms_ssim = np.prod(np.array(mssim) ** np.array(weights))
    return ms_ssim


def calculate_emd(img1, img2, bins=256):
    # For color images, compute EMD for each channel and average the results.
    if img1.ndim == 3:
        emd_vals = []
        for c in range(img1.shape[2]):
            hist1, _ = np.histogram(img1[:,:,c].flatten(), bins=bins, density=True)
            hist2, _ = np.histogram(img2[:,:,c].flatten(), bins=bins, density=True)
            bin_centers = np.linspace(0, 255, bins)
            emd_val = wasserstein_distance(bin_centers, bin_centers, hist1, hist2)
            emd_vals.append(emd_val)
        return np.mean(emd_vals)
    else:
        hist1, _ = np.histogram(img1.flatten(), bins=bins, density=True)
        hist2, _ = np.histogram(img2.flatten(), bins=bins, density=True)
        bin_centers = np.linspace(0, 255, bins)
        return wasserstein_distance(bin_centers, bin_centers, hist1, hist2)

def calculate_mse(img1, img2):
    return np.mean((img1.astype("float") - img2.astype("float"))**2)

def calculate_pcc(img1, img2):
    img1_flat = img1.flatten()
    img2_flat = img2.flatten()
    if np.std(img1_flat) == 0 or np.std(img2_flat) == 0:
        return 0
    corr_matrix = np.corrcoef(img1_flat, img2_flat)
    return corr_matrix[0, 1]

def compare_images(img1_path, img2_path):
    # Load images in color (RGB)
    img1 = np.array(Image.open(img1_path).convert('RGB'))
    img2 = np.array(Image.open(img2_path).convert('RGB'))
    
    # Ensure both images have the same dimensions.
    if img1.shape != img2.shape:
        if img1.shape[0]*img1.shape[1] > img2.shape[0]*img2.shape[1]:
            img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
        else:
            img1 = cv2.resize(img1, (img2.shape[1], img2.shape[0]))
    
    return {
        'MS-SSIM': calculate_ms_ssim(img1, img2),
        'PSNR': psnr(img1, img2, data_range=255),
        'EMD': calculate_emd(img1, img2),
        'MSE': calculate_mse(img1, img2),
        'PCC': calculate_pcc(img1, img2)
    }


In [ ]:
# scores = compare_images('output/AperioCS2.ome.tif', 'output/HamamatsuS360.ome.tif'),
# print(scores)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# -----------------------------
# Helper function to subdivide an image into full tiles
# -----------------------------
def get_tiles(image, tile_size=8192):
    """
    Returns a list of full tiles (tile_size x tile_size) from the image.
    Tiles on the borders that are smaller than tile_size are ignored.
    """
    tiles = []
    h, w = image.shape[0:2]
    num_tiles_y = h // tile_size
    num_tiles_x = w // tile_size
    for i in range(num_tiles_y):
        for j in range(num_tiles_x):
            y = i * tile_size
            x = j * tile_size
            tile = image[y:y+tile_size, x:x+tile_size]
            if tile.shape[0] == tile_size and tile.shape[1] == tile_size:
                tiles.append(tile)
    return tiles

# -----------------------------
# Main processing: load image pairs, compute metrics over tiles, and save results
# -----------------------------
def process_image_pairs(image_pairs, tile_size=8192, output_csv='quantitative_metrics/metrics_results.csv'):
    results = []
    for img1_path, img2_path in image_pairs:
        print(f"Processing pair:\n  {img1_path}\n  {img2_path}")
        # Load the first image (pixel size = 0.5µm) as a full-resolution RGB image.
        img1 = np.array(Image.open(img1_path).convert('RGB'))
        print("First loaded")
        
        # Load the second image (pixel size = 0.325µm) as a full-resolution RGB image.
        img2_full = np.array(Image.open(img2_path).convert('RGB'))
        print("Second loaded")
        
        # Resample the second image to 0.5µm pixel size.
        scale_factor = 0.325 / 0.5  # ≈0.65
        new_width = int(img2_full.shape[1] * scale_factor)
        new_height = int(img2_full.shape[0] * scale_factor)
        img2 = cv2.resize(img2_full, (new_width, new_height), interpolation=cv2.INTER_AREA)
        print("Second resized")
        
        # Ensure both images have identical dimensions.
        if img1.shape != img2.shape:
            img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_AREA)
        
        # Downsample images to 1024x1024 for visualization.
        target_size = (1024, 1024)
        img1_down = cv2.resize(img1, target_size, interpolation=cv2.INTER_AREA)
        img2_down = cv2.resize(img2, target_size, interpolation=cv2.INTER_AREA)

        # Plot images side by side.
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(img1_down)
        axes[0].set_title('Downsampled Image 1')
        axes[0].axis('off')

        axes[1].imshow(img2_down)
        axes[1].set_title('Downsampled Image 2')
        axes[1].axis('off')

        plt.show()
        
        # Subdivide both images into 8192×8192 tiles (ignoring border tiles).
        tiles1 = get_tiles(img1, tile_size)
        print("First tiled")
        tiles2 = get_tiles(img2, tile_size)
        print("Second tiled")
        num_tiles = min(len(tiles1), len(tiles2))
        if num_tiles == 0:
            print(f"Warning: No full {tile_size}x{tile_size} tiles found. Skipping pair.")
            continue
                    
        # Compute metrics on each tile pair.
        metrics_list = {
            'MS-SSIM': [],
            'PSNR': [],
            'EMD': [],
            'MSE': [],
            'PCC': []
        }
        for i in range(num_tiles):
            print(f"Calculating metrics tile {i}/{num_tiles}")
            t1 = tiles1[i]
            t2 = tiles2[i]
            metrics_list['MS-SSIM'].append(calculate_ms_ssim(t1, t2))
            metrics_list['PSNR'].append(psnr(t1, t2, data_range=255))
            metrics_list['EMD'].append(calculate_emd(t1, t2))
            metrics_list['MSE'].append(calculate_mse(t1, t2))
            metrics_list['PCC'].append(calculate_pcc(t1, t2))
            
        # Average metrics over all tiles for the current image pair.
        result = {
            'pair': os.path.basename(img1_path),
            'MS-SSIM': np.mean(metrics_list['MS-SSIM']),
            'PSNR': np.mean(metrics_list['PSNR']),
            'EMD': np.mean(metrics_list['EMD']),
            'MSE': np.mean(metrics_list['MSE']),
            'PCC': np.mean(metrics_list['PCC'])
        }
        print(result)
        results.append(result)
        
        # Write intermediate results to CSV after processing this pair.
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False)
        print(f"Intermediate results saved to {output_csv}\n")
    
    print(f"Final results saved to {output_csv}")
    return df



# Define the list of image pairs (first column, second column).
image_pairs = [
    ('CRC2/H&E/data_CRC01_P37_S29_A24_C59kX_E15_20220106_014304_946511-zlib.ome.tiff', 'CRC2/data_CRC01_18459_LSP10353_US_SCAN_OR_001__093059-registered.ome.tif'),
    ('CRC2/H&E/data_CRC02_P37_S30_A24_C59kX_E15_20220106_014319_409148-zlib.ome.tiff', 'CRC2/data_CRC02_18459_LSP10364_US_SCAN_OR_001__092347-registered.ome.tif'),
    ('CRC2/H&E/data_CRC03_P37_S31_A24_C59kX_E15_20220106_014409_014236-zlib.ome.tiff', 'CRC2/data_CRC03_18459_LSP10375_US_SCAN_OR_001__092147-registered.ome.tif'),
    ('CRC2/H&E/data_CRC04_P37_S32_A24_C59kX_E15_20220106_014630_553652-zlib.ome.tiff', 'CRC2/data_CRC04_18459_LSP10388_US_SCAN_OR_001__091155-registered.ome.tif'),
    ('CRC2/H&E/data_CRC05_P37_S33_A24_C59kX_E15_20220107_180446_881530-zlib.ome.tiff', 'CRC2/data_CRC05_18459_LSP10397_US_SCAN_OR_001__091631-registered.ome.tif'),
    ('CRC2/H&E/data_CRC06_P37_S34_A24_C59kX_E15_20220107_202112_212579-zlib.ome.tiff', 'CRC2/data_CRC06_18459_LSP10408_US_SCAN_OR_001__092559-registered.ome.tif'),
    ('CRC2/H&E/data_CRC07_P37_S35_A24_C59kX_E15_20220108_012037_490594-zlib.ome.tiff', 'CRC2/data_CRC07_18459_LSP10419_US_SCAN_OR_001__090907-registered.ome.tif'),
    ('CRC2/H&E/data_CRC08_P37_S57_Full_A24_C59nX_E15_20220224_011032_774034-zlib.ome.tiff', 'CRC2/data_CRC08_19510_C8_US_SCAN_OR_001__150825-registered.ome.tif'),
    ('CRC2/H&E/data_CRC09_P37_S37_A24_C59kX_E15_20220108_012113_953544-zlib.ome.tiff', 'CRC2/data_CRC09_18459_LSP10441_US_SCAN_OR_001__091844-registered.ome.tif'),
    ('CRC2/H&E/data_CRC10_P37_S38_A24_C59kX_E15_20220108_012130_664519-zlib.ome.tiff', 'CRC2/data_CRC10_18459_LSP10452_US_SCAN_OR_001__091355-registered.ome.tif'),
    ('CRC2/H&E/data_CRC11_P37_S43_Full_A24_C59mX_E15_20220128_171510_544056-zlib.ome.tiff', 'CRC2/data_CRC11_19510_C11_US_SCAN_OR_001__151039-registered.ome.tif'),
    ('CRC2/H&E/data_CRC12_P37_S44_Full_A24_C59mX_E15_20220128_171448_903938-zlib.ome.tiff', 'CRC2/data_CRC12_19510_C12_US_SCAN_OR_001__151249-registered.ome.tif'),
    ('CRC2/H&E/data_CRC13_P37_S45_Full_A24_C59mX_E15_20220128_171409_633341-zlib.ome.tiff', 'CRC2/data_CRC13_19510_C13_US_SCAN_OR_001__151503-registered.ome.tif'),
    ('CRC2/H&E/data_CRC14_P37_S46_Full_A24_C59mX_E15_20220128_013821_398547-zlib.ome.tiff', 'CRC2/data_CRC14_19510_C14_US_SCAN_OR_001__151737-registered.ome.tif'),
    ('CRC2/H&E/data_CRC15_P37_S47_Full_A24_C59mX_E15_20220128_020654_901143-zlib.ome.tiff', 'CRC2/data_CRC15_19510_C15_US_SCAN_OR_001__152234-registered.ome.tif'),
    ('CRC2/H&E/data_CRC16_P37_S48_Full_A24_C59mX_E15_20220129_015105_865195-zlib.ome.tiff', 'CRC2/data_CRC16_19510_C16_US_SCAN_OR_001__152020-registered.ome.tif'),
    ('CRC2/H&E/data_CRC17_P37_S49_Full_A24_C59mX_E15_20220129_015121_911264-zlib.ome.tiff', 'CRC2/data_CRC17_19510_C17_US_SCAN_OR_001__152525-registered.ome.tif'),
    ('CRC2/H&E/data_CRC18_P37_S50_Full_A24_C59mX_E15_20220129_015242_755602-zlib.ome.tiff', 'CRC2/data_CRC18_19510_C18_US_SCAN_OR_001__152757-registered.ome.tif'),
    ('CRC2/H&E/data_CRC19_P37_S51_Full_A24_C59mX_E15_20220129_015300_669681-zlib.ome.tiff', 'CRC2/data_CRC19_19510_C19_US_SCAN_OR_001__153041-registered.ome.tif'),
    ('CRC2/H&E/data_CRC20_P37_S52_Full_A24_C59mX_E15_20220129_015324_574779-zlib.ome.tiff', 'CRC2/data_CRC20_19510_C20_US_SCAN_OR_001__153341-registered.ome.tif'),
    ('CRC2/H&E/data_CRC21_P37_S58_Full_A24_C59nX_E15_20220224_011058_014787-zlib.ome.tiff', 'CRC2/data_CRC21_19510_C21_US_SCAN_OR_001__153607-registered.ome.tif'),
    ('CRC2/H&E/data_CRC22_P37_S59_Full_A24_C59nX_E15_20220224_011113_455637-zlib.ome.tiff', 'CRC2/data_CRC22_19510_C22_US_SCAN_OR_001__092420-registered.ome.tif'),
    ('CRC2/H&E/data_CRC23_P37_S60_Full_A24_C59nX_E15_20220224_011127_971497-zlib.ome.tiff', 'CRC2/data_CRC23_19510_C23_US_SCAN_OR_001__154147-registered.ome.tif'),
    ('CRC2/H&E/data_CRC24_P37_S61_Full_A24_C59nX_E15_20220224_011149_079291-zlib.ome.tiff', 'CRC2/data_CRC24_19510_C24_US_SCAN_OR_001__091904-registered.ome.tif'),
    ('CRC2/H&E/data_CRC25_P37_S62_Full_A24_C59nX_E15_20220224_011204_784145-zlib.ome.tiff', 'CRC2/data_CRC25_19510_C25_US_SCAN_OR_001__154712-registered.ome.tif'),
    ('CRC2/H&E/data_CRC26_P37_S63_Full_A24_C59nX_E15_20220224_011246_458738-zlib.ome.tiff', 'CRC2/data_CRC26_19510_C26_US_SCAN_OR_001__092131-registered.ome.tif'),
    ('CRC2/H&E/data_CRC27_P37_S64_Full_A24_C59nX_E15_20220224_011259_841605-zlib.ome.tiff', 'CRC2/data_CRC27_19510_C27_US_SCAN_OR_001__155205-registered.ome.tif'),
    ('CRC2/H&E/data_CRC28_P37_S65_Full_A24_C59nX_E15_20220224_011333_386280-zlib.ome.tiff', 'CRC2/data_CRC28_19510_C28_US_SCAN_OR_001__155413-registered.ome.tif'),
    ('CRC2/H&E/data_CRC29_P37_S66_Full_A24_C59nX_E15_20220224_011348_519133-zlib.ome.tiff', 'CRC2/data_CRC29_19510_C29_US_SCAN_OR_001__155859-registered.ome.tif'),
    ('CRC2/H&E/data_CRC30_P37_S67_Full_A24_C59nX_E15_20220224_011408_506939-zlib.ome.tiff', 'CRC2/data_CRC30_19510_C30_US_SCAN_OR_001__155702-registered.ome.tif'),
    ('CRC2/H&E/data_CRC31_P37_S74_Full_A24_C59qX_E15_20220302_234837_137590-zlib.ome.tiff', 'CRC2/data_CRC31_19510_C31_US_SCAN_OR_001__160203-registered.ome.tif'),
    ('CRC2/H&E/data_CRC32_P37_S75_Full_A24_C59qX_E15_20220302_235001_586560-zlib.ome.tiff', 'CRC2/data_CRC32_19510_C32_US_SCAN_OR_001__160434-registered.ome.tif'),
    ('CRC2/H&E/data_CRC33_01_P37_S76_01_A24_C59qX_E15_20220302_235136_561323-zlib.ome.tiff', 'CRC2/data_CRC33_01_19510_C33_US_SCAN_OR_001__160715-2-registered.ome.tif'),
    ('CRC2/H&E/data_CRC33_02_P37_S76_02_A24_C59qX_E15_20220302_235158_533766-zlib.ome.tiff', 'CRC2/data_CRC33_02_19510_C33_US_SCAN_OR_001__160715-registered.ome.tif'),
    ('CRC2/H&E/data_CRC34_P37_S77_Full_A24_C59qX_E15_20220302_235222_359806-zlib.ome.tiff', 'CRC2/data_CRC34_19510_C34_US_SCAN_OR_001__160949-registered.ome.tif'),
    ('CRC2/H&E/data_CRC35_P37_S78_Full_A24_C59qX_E15_20220302_235239_498836-zlib.ome.tiff', 'CRC2/data_CRC35_19510_C35_US_SCAN_OR_001__161209-registered.ome.tif'),
    ('CRC2/H&E/data_CRC36_P37_S79_Full_A24_C59qX_E15_20220302_235254_496641-zlib.ome.tiff', 'CRC2/data_CRC36_19510_C36_US_SCAN_OR_001__161442-registered.ome.tif'),
    ('CRC2/H&E/data_CRC37_P37_S80_Full_A24_C59qX_E15_20220307_235159_333000-zlib.ome.tiff', 'CRC2/data_CRC37_19510_C37_US_SCAN_OR_001__161733-registered.ome.tif'),
    ('CRC2/H&E/data_CRC38_P37_S81_Full_A24_C59qX_E15_20220302_235331_704703-zlib.ome.tiff', 'CRC2/data_CRC38_19510_C38_US_SCAN_OR_001__162018-registered.ome.tif'),
    ('CRC2/H&E/data_CRC39_P37_S82_Full_A24_C59qX_E15_20220304_200614_832683-zlib.ome.tiff', 'CRC2/data_CRC39_19510_C39_US_SCAN_OR_001__162343-registered.ome.tif'),
    ('CRC2/H&E/data_CRC40_P37_S83_Full_A24_C59qX_E15_20220304_200429_490805-zlib.ome.tiff', 'CRC2/data_CRC40_19510_P37-S83_C40_US_SCAN_OR_001__163912-registered.ome.tif'),

]

# Process image pairs and save metrics to CSV.
process_image_pairs(image_pairs, tile_size=8192, output_csv='quantitative_metrics/metrics_results.csv')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def get_violin_line_x_limits(ax, y_val):
    """
    Compute the x-coordinate intersections of the violin's outline with a horizontal line at y = y_val.
    Returns the minimum and maximum x-values of the intersections.
    """
    intersections = []
    for path in ax.collections[0].get_paths():
        vertices = path.vertices
        n = len(vertices)
        for i in range(n):
            v1 = vertices[i]
            v2 = vertices[(i + 1) % n]
            y1, y2 = v1[1], v2[1]
            # Check if the horizontal line at y_val crosses this segment:
            if (y1 - y_val) * (y2 - y_val) < 0:
                x1, x2 = v1[0], v2[0]
                # Linear interpolation:
                t = (y_val - y1) / (y2 - y1)
                x_intersect = x1 + t * (x2 - x1)
                intersections.append(x_intersect)
            elif y1 == y_val:
                intersections.append(v1[0])
    if intersections:
        return min(intersections), max(intersections)
    else:
        return None, None

# Load CSV data
df = pd.read_csv('quantitative_metrics/metrics_results.csv')
metrics = ['MS-SSIM', 'PSNR', 'EMD', 'MSE', 'PCC']

# Set a clean Seaborn theme
sns.set_theme(style="white")
colors = sns.color_palette("Set3", len(metrics))

fig, axes = plt.subplots(1, len(metrics), figsize=(10, 5))

for ax, metric, color in zip(axes, metrics, colors):
    # Plot the violin plot without the inner boxplot
    sns.violinplot(data=df, y=metric, ax=ax, color=color, inner=None)
    ax.set_title(metric)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.grid(False)
    
    # Compute mean and standard deviation for the current metric
    mean_val = df[metric].mean()
    sd_val = df[metric].std()
    
    # Get the x-limits for a horizontal line at the mean
    x_min, x_max = get_violin_line_x_limits(ax, mean_val)
    if x_min is None or x_max is None:
        x_min, x_max = 0.25, 0.75

    # Get the x-limits for horizontal lines at mean+SD and mean-SD
    x_min_upper, x_max_upper = get_violin_line_x_limits(ax, mean_val + sd_val)
    if x_min_upper is None or x_max_upper is None:
        x_min_upper, x_max_upper = x_min, x_max
    x_min_lower, x_max_lower = get_violin_line_x_limits(ax, mean_val - sd_val)
    if x_min_lower is None or x_max_lower is None:
        x_min_lower, x_max_lower = x_min, x_max
        
    # Draw a solid horizontal line at the mean, spanning exactly the violin outline
    ax.plot([x_min, x_max], [mean_val, mean_val], color='black', linestyle='-', linewidth=1)
    # Draw dashed lines at mean+SD and mean-SD using the corresponding x-limits
    ax.plot([x_min_upper, x_max_upper], [mean_val + sd_val, mean_val + sd_val], 
            color='grey', linestyle='--', linewidth=1)
    ax.plot([x_min_lower, x_max_lower], [mean_val - sd_val, mean_val - sd_val], 
            color='grey', linestyle='--', linewidth=1)
    
    # Format annotation depending on the metric
    if metric == 'PSNR':
        annotation = f'{mean_val:.1f} ± {sd_val:.1f}'
    elif metric == 'MSE':
        annotation = f'{int(mean_val)} ± {int(sd_val)}'
    else:
        annotation = f'{mean_val:.2f} ± {sd_val:.2f}'
    
    # Annotate the line with the text (only the numerical values)
    ax.text((x_min + x_max) / 2, mean_val, annotation,
            ha='center', va='bottom', color='black')

fig.suptitle('Violin Plot of Metrics Across Image Pairs', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()
